In [1]:
import os
import cv2
import numpy as np
from PIL import Image
import imagehash
from pathlib import Path
import logging
from tqdm import tqdm
import requests
import time

In [2]:
# Configuración
PIXABAY_API_KEY = "49861357-734e1eca2425b5e98b167e58f"  # Reemplaza con tu clave de API de Pixabay
CATEGORIES = ["flowers", "mountains", "animals", "everyday objects", "urban environments"]
IMAGES_PER_CATEGORY = 1200  # Descargar más para tener margen
FINAL_IMAGES_PER_CATEGORY = 1000  # Objetivo final
TEMP_DIR = "C:/Users/ctorr/OneDrive/Documentos/Arturo/Arturo HW ITESO/6to semestre/Modelos no lineales/GANs/data/raw"  # Directorio temporal
OUTPUT_DIR = "C:/Users/ctorr/OneDrive/Documentos/Arturo/Arturo HW ITESO/6to semestre/Modelos no lineales/GANs/data/clean"  # Directorio final
LOG_FILE = "dataset_preparation.log"
MIN_SIZE = (100, 100)  # Tamaño mínimo
TARGET_SIZE = (256, 256)  # Tamaño objetivo
SATURATION_THRESHOLD = 0.1  # Umbral de saturación
PHASH_THRESHOLD = 5  # Umbral de distancia para pHash
REQUESTS_PER_CYCLE = 100  # Máximo de solicitudes por ciclo
PAUSE_SECONDS = 60  # Pausa entre ciclos

In [3]:
# Configurar logging
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
def download_images(category, temp_dir, max_images):
    """Descarga imágenes de Pixabay usando la API."""
    category_dir = os.path.join(temp_dir, category.replace(" ", "_"))
    Path(category_dir).mkdir(parents=True, exist_ok=True)
    
    # Mostrar ruta de descarga
    print(f"\nDescargando imágenes de '{category}' en: {os.path.abspath(category_dir)}")
    logging.info(f"Iniciando descarga para '{category}' en: {os.path.abspath(category_dir)}")
    
    count = 0
    page = 1
    requests_made = 0
    
    while count < max_images:
        # Construir URL de la API
        url = f"https://pixabay.com/api/?key={PIXABAY_API_KEY}&q={category}&image_type=photo&per_page=100&page={page}"
        
        try:
            # Realizar solicitud
            response = requests.get(url, timeout=10)
            requests_made += 1
            
            if response.status_code != 200:
                logging.warning(f"Error en la solicitud para {category}, página {page}: {response.status_code}")
                break
            
            data = response.json()
            hits = data.get('hits', [])
            if not hits:
                logging.info(f"No más imágenes para {category} en la página {page}")
                break
            
            # Descargar imágenes
            for i, hit in enumerate(tqdm(hits, desc=f"Descargando {category}, página {page}")):
                if count >= max_images:
                    break
                img_url = hit['largeImageURL']
                img_path = os.path.join(category_dir, f"{category.replace(' ', '_')}_{count+1:04d}.jpg")
                try:
                    img_response = requests.get(img_url, stream=True, timeout=10)
                    if img_response.status_code == 200:
                        with open(img_path, 'wb') as f:
                            f.write(img_response.content)
                        count += 1
                    time.sleep(0.2)  # Pausa breve entre descargas
                except Exception as e:
                    logging.warning(f"Error al descargar {img_url}: {str(e)}")
            
            # Controlar límite de solicitudes
            if requests_made >= REQUESTS_PER_CYCLE:
                logging.info(f"Límite de {REQUESTS_PER_CYCLE} solicitudes alcanzado. Pausando {PAUSE_SECONDS} segundos.")
                time.sleep(PAUSE_SECONDS)
                requests_made = 0
            
            page += 1
        
        except Exception as e:
            logging.warning(f"Error en la solicitud para {category}, página {page}: {str(e)}")
            break
    
    logging.info(f"Descargadas {count} imágenes para {category}")
    return count

In [5]:
def is_image_corrupt(file_path):
    """Verifica si la imagen está corrupta."""
    try:
        img = Image.open(file_path)
        img.verify()
        img = Image.open(file_path)
        img.load()
        return False
    except Exception as e:
        logging.warning(f"Imagen corrupta: {file_path}, Error: {str(e)}")
        return True

def is_low_resolution(file_path):
    """Verifica si la imagen tiene resolución muy baja."""
    try:
        img = Image.open(file_path)
        width, height = img.size
        if width < MIN_SIZE[0] or height < MIN_SIZE[1]:
            logging.info(f"Imagen de baja resolución descartada: {file_path}, Tamaño: {width}x{height}")
            return True
        return False
    except:
        return True

def is_monochromatic(file_path):
    """Verifica si la imagen es casi monocromática."""
    try:
        img = cv2.imread(file_path)
        if img is None:
            return True
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        saturation = hsv[:, :, 1].mean() / 255.0
        if saturation < SATURATION_THRESHOLD:
            logging.info(f"Imagen monocromática descartada: {file_path}, Saturación: {saturation:.3f}")
            return True
        return False
    except Exception as e:
        logging.warning(f"Error al verificar monocromía: {file_path}, Error: {str(e)}")
        return True

def get_phash(file_path):
    """Calcula el hash perceptual (pHash)."""
    try:
        img = Image.open(file_path)
        return imagehash.phash(img)
    except:
        return None

def remove_duplicates(file_paths):
    """Elimina imágenes duplicadas basadas en pHash."""
    hashes = {}
    keep_files = []
    for file_path in tqdm(file_paths, desc="Buscando duplicados"):
        phash = get_phash(file_path)
        if phash is None:
            logging.warning(f"No se pudo calcular pHash: {file_path}")
            continue
        duplicate = False
        for existing_hash in hashes:
            if phash - existing_hash <= PHASH_THRESHOLD:
                logging.info(f"Duplicado encontrado: {file_path}, Similar a: {hashes[existing_hash]}")
                duplicate = True
                break
        if not duplicate:
            hashes[phash] = file_path
            keep_files.append(file_path)
    return keep_files

In [6]:
def process_image(file_path, output_dir, filename):
    """Procesa una imagen: redimensiona y convierte a JPG."""
    try:
        img = Image.open(file_path).convert('RGB')
        img = img.resize(TARGET_SIZE, Image.LANCZOS)
        output_path = os.path.join(output_dir, filename)
        img.save(output_path, 'JPEG', quality=95)
        return True
    except Exception as e:
        logging.warning(f"Error al procesar imagen: {file_path}, Error: {str(e)}")
        return False

def process_category(category, temp_dir, output_dir, max_images):
    """Procesa una categoría completa."""
    category_temp_dir = os.path.join(temp_dir, category.replace(" ", "_"))
    category_output_dir = os.path.join(output_dir, category.replace(" ", "_"))
    Path(category_output_dir).mkdir(parents=True, exist_ok=True)
    
    # Mostrar rutas
    print(f"\nProcesando imágenes de '{category}':")
    print(f" - Origen: {os.path.abspath(category_temp_dir)}")
    print(f" - Destino: {os.path.abspath(category_output_dir)}")
    logging.info(f"Procesando '{category}': Origen: {category_temp_dir}, Destino: {category_output_dir}")
    
    # Obtener lista de imágenes descargadas
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    file_paths = [os.path.join(category_temp_dir, f) for f in os.listdir(category_temp_dir) 
                  if f.lower().endswith(image_extensions)]
    
    logging.info(f"Se encontraron {len(file_paths)} imágenes para {category}")
    
    # Filtrar imágenes corruptas y de baja resolución
    valid_files = []
    for file_path in tqdm(file_paths, desc=f"Filtrando {category} - Corruptas/Baja resolución"):
        if not (is_image_corrupt(file_path) or is_low_resolution(file_path)):
            valid_files.append(file_path)
    
    # Filtrar imágenes monocromáticas
    non_monochromatic_files = []
    for file_path in tqdm(valid_files, desc=f"Filtrando {category} - Monocromáticas"):
        if not is_monochromatic(file_path):
            non_monochromatic_files.append(file_path)
    
    # Eliminar duplicados
    unique_files = remove_duplicates(non_monochromatic_files)
    
    # Procesar imágenes (redimensionar y convertir a JPG)
    processed_count = 0
    for i, file_path in enumerate(tqdm(unique_files, desc=f"Procesando {category}")):
        if processed_count >= max_images:
            break
        filename = f"{category.replace(' ', '_')}_{i+1:04d}.jpg"
        if process_image(file_path, category_output_dir, filename):
            processed_count += 1
    
    logging.info(f"Procesadas {processed_count} imágenes para {category}")
    return processed_count

In [ ]:
def main():
    # Crear directorios
    Path(TEMP_DIR).mkdir(parents=True, exist_ok=True)
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    
    # Descargar imágenes para cada categoría
    for category in CATEGORIES:
        count = download_images(category, TEMP_DIR, IMAGES_PER_CATEGORY)
        print(f"Descargadas {count} imágenes para {category}")
    
    # Procesar imágenes para cada categoría
    for category in CATEGORIES:
        count = process_category(category, TEMP_DIR, OUTPUT_DIR, FINAL_IMAGES_PER_CATEGORY)
        print(f"Procesadas {count} imágenes para {category}")
        
    print(f"Procesamiento completado. Revisa el log en {LOG_FILE}")

if __name__ == "__main__":
    main()


Descargando imágenes de 'flowers' en: C:\Users\ctorr\OneDrive\Documentos\Arturo\Arturo HW ITESO\6to semestre\Modelos no lineales\GANs\data\raw\flowers


Descargando flowers, página 6: 100%|██████████| 100/100 [00:51<00:00,  1.94it/s]


Descargadas 600 imágenes para flowers

Descargando imágenes de 'mountains' en: C:\Users\ctorr\OneDrive\Documentos\Arturo\Arturo HW ITESO\6to semestre\Modelos no lineales\GANs\data\raw\mountains


Descargando mountains, página 6: 100%|██████████| 100/100 [01:13<00:00,  1.36it/s]


Descargadas 600 imágenes para mountains

Descargando imágenes de 'animals' en: C:\Users\ctorr\OneDrive\Documentos\Arturo\Arturo HW ITESO\6to semestre\Modelos no lineales\GANs\data\raw\animals


Descargando animals, página 4:  69%|██████▉   | 69/100 [00:48<00:21,  1.46it/s]